# Distillation 

Ce a pour but d'entrainer un MPS multicouche à l'aide d'un réseau complètement connecté

In [1]:
%pip install "git+https://github.com/jemisjoky/TorchMPS.git"
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jemisjoky/TorchMPS.git to /tmp/pip-req-build-ejwe82f5
  Running command git clone --filter=blob:none --quiet https://github.com/jemisjoky/TorchMPS.git /tmp/pip-req-build-ejwe82f5
  Resolved https://github.com/jemisjoky/TorchMPS.git to commit 6c0bc1a8e2c15acba8570ca9ffe2b4a0c7135165
  Preparing metadata (setup.py) ... done
  Created wheel for torchmps: filename=torchmps-0.1.0-py3-none-any.whl size=59679 sha256=507e63d604c34f834668d9acad1013e10d83c0a35f2bb0c2e5931fbef1fbb9ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-xwgp2r7j/wheels/73/fe/06/7ff82b6d4fdab89247f7e9a997e88679ee38603c785bb2fa10
Successfully built torchmps
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 20.4 MB/s eta 0:00:00


In [2]:
# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train = 200
nb_test = 50
chosen_bs = 100
input_size = 28*28
nb_classes = 10

# Teacher hyperparameters
chosen_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_epochs_fcnn = 20
hidden_size = 70
chosen_loss = nn.CrossEntropyLoss()
#Optimizer parameters
chosen_lr = 0.001
chosen_momentum = 0.9

# Student hyperparameters
# MPS parameters
bond_dim = 20
adaptive_mode = False
periodic_bc = False
feature_map = lambda x : torch.tensor([1, x]).to(chosen_device)
feature_dim = 2
# Training parameters
gauss_epochs = 5 # number of epochs with added gaussian noise
gn_var = 0.3 #added gaussian noise variance
gn_mean = 0 #added gaussian noise mean
n_epochs_lmps = 30
mps_learn_rate = 0.0001
mps_reg = 0.0
mps_chosen_loss = nn.CrossEntropyLoss().to(chosen_device)

In [3]:
# Import the mnist dataset
train_set = torchvision.datasets.MNIST(root = './datasets', train = True,   
    transform = transforms.ToTensor(),  download = True )

train_subset = torch.utils.data.SubsetRandomSampler(range(nb_train))

train_iterator = torch.utils.data.DataLoader(dataset = train_set, 
    sampler = train_subset, batch_size=chosen_bs)


test_set = torchvision.datasets.MNIST(root = './datasets',
 train = False, transform = transforms.ToTensor(),  download = True)

test_subset = torch.utils.data.SubsetRandomSampler(range(nb_test))

test_iterator = torch.utils.data.DataLoader(dataset = test_set, 
    sampler = test_subset, batch_size = chosen_bs)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/train-images-idx3-ubyte.gz to ./datasets/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/train-labels-idx1-ubyte.gz to ./datasets/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to ./datasets/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./datasets/MNIST/raw



In [4]:

# Create the fcnn class
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.relu = nn.ReLU()
        self.lin1 = nn.Linear(784, hidden_size)
        self.lin2 = nn.Linear(hidden_size, hidden_size)
        self.lin3 = nn.Linear(hidden_size, hidden_size)
        self.lin4 = nn.Linear(hidden_size, hidden_size)
        self.lin5 = nn.Linear(hidden_size, hidden_size)
        self.lin6 = nn.Linear(hidden_size, 10)

    def middleforward(self, x):
        y = self.lin1(x)
        y = self.relu(y)
        y = self.lin2(y)
        y = self.relu(y)
        y = self.lin3(y)
        y = self.relu(y)
        return y

    def forward(self, x):
        y = self.lin1(x)
        y = self.relu(y)
        y = self.lin2(y)
        y = self.relu(y)
        y = self.lin3(y)
        y = self.relu(y)
        y = self.lin4(y)
        y = self.relu(y)
        y = self.lin5(y)
        y = self.relu(y)
        y = self.lin6(y)
        y = self.relu(y)
        return y

#Instantiate and put the model on the chosen device
fcnn_teacher = FCNN().to(chosen_device)

#Instantiate the optimizer
optimizer = torch.optim.Adam(fcnn_teacher.parameters())

#Training loop
for epoch in range(n_epochs_fcnn):
    for (x_mb, y_mb) in train_iterator:
        # Reshape the train_tuple and put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Foward propagation
        y_hat_mb = fcnn_teacher(x_mb)
        loss = chosen_loss(y_hat_mb, y_mb)
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    print(loss.item())

# get accuracy
# Get the validation set classification accuracy
teacher_acc_score = 0
teacher_acc_metric = MulticlassAccuracy(num_classes=nb_classes).to(chosen_device)
for (x_mb, y_mb) in test_iterator:
    x_mb = x_mb.reshape(-1, 784).to(chosen_device)
    y_mb = y_mb.to(chosen_device)
    # add the number of datapoints we classified right
    teacher_acc_score += x_mb.size()[0] * teacher_acc_metric( fcnn_teacher(x_mb), y_mb )
print("The teacher's accuracy score is:")
print(teacher_acc_score / nb_test) #divide by total size



2.297206163406372
2.2958199977874756
2.2882132530212402
2.2859201431274414
2.2744710445404053
2.2674965858459473
2.257532835006714
2.225705146789551
2.225538730621338
2.197650909423828
2.1052205562591553
2.059401273727417
2.002795457839966
1.9138116836547852
1.8335964679718018
1.7136547565460205
1.6382030248641968
1.3669780492782593
1.479541301727295
1.3056585788726807
The teacher's accuracy score is:
tensor(0.4972, device='cuda:0')


In [5]:

# Initialize the MPS modules
student_mps1 = MPS(
    input_dim = input_size,
    feature_dim = feature_dim,
    output_dim = hidden_size,
    bond_dim = bond_dim,
    adaptive_mode = adaptive_mode,
    periodic_bc = periodic_bc,
).to(chosen_device)
student_mps1.register_feature_map(feature_map)

student_mps2 = MPS(
    input_dim = hidden_size, #crucial setting
    feature_dim = feature_dim,
    output_dim = nb_classes,
    bond_dim = bond_dim,
    adaptive_mode = adaptive_mode,
    periodic_bc = periodic_bc,
).to(chosen_device)
student_mps2.register_feature_map(feature_map)

def student(x):
    return student_mps2( student_mps1(x) )


# Instantiate the optimizer and softmax
lmps1_optimizer = torch.optim.Adam(student_mps1.parameters(), lr = mps_learn_rate,
                                  weight_decay = mps_reg)

lmps2_optimizer = torch.optim.Adam(student_mps2.parameters(), lr = mps_learn_rate,
                                  weight_decay = mps_reg)

softmax = nn.Softmax(dim=1)

# Training loop for the student_mps1
for epoch in range(n_epochs_lmps):
    for (x_mb, _) in train_iterator:
        # Reshape put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        # Added gaussian noise
        if epoch > (n_epochs_lmps - gauss_epochs):
          x_mb = x_mb + torch.randn(size=x_mb.size()).to(chosen_device)

        # Get softmax of output of teacher in middlelayer!
        y_mb = (softmax(fcnn_teacher.middleforward(x_mb))).to(chosen_device)

        # Foward propagation
        y_hat_mb = student_mps1(x_mb)
        loss = mps_chosen_loss(y_hat_mb, y_mb)

        # Backpropagation
        loss.backward()
        lmps1_optimizer.step()
        lmps1_optimizer.zero_grad()

    print(loss.item())

# Training loop for the student_mps2
for epoch in range(n_epochs_lmps):
    for (x_mb, _) in train_iterator:
        # Reshape put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        # Added gaussian noise
        if epoch > (n_epochs_lmps - gauss_epochs):
          x_mb = x_mb + torch.randn(size=x_mb.size()).to(chosen_device)

        # Get softmax of output of teacher
        y_mb = ( softmax(fcnn_teacher(x_mb)) ).to(chosen_device)

        # Foward propagation
        y_hat_mb = student_mps1(x_mb)
        # Apply second forward propagation on first output
        y_hat_mb = student_mps2(y_hat_mb)
        loss = mps_chosen_loss(y_hat_mb, y_mb)

        # Backpropagation
        loss.backward()
        lmps2_optimizer.step()
        lmps2_optimizer.zero_grad()

    print(loss.item())



# Get the validation set classification accuracy
student_acc_score = 0
student_acc_metric = MulticlassAccuracy(num_classes=nb_classes).to(chosen_device)
for (x_mb, y_mb) in test_iterator:
    x_mb = x_mb.reshape(-1, 784).to(chosen_device)
    y_mb = y_mb.to(chosen_device)
    # add the number of datapoints we classified right
    student_acc_score += x_mb.size()[0] * student_acc_metric( student(x_mb), y_mb )
print("Student_acc_score:")
print(student_acc_score / nb_test) #divide by total size


4.2484259605407715
4.247060298919678
4.217930793762207
3.83778977394104
3.7018163204193115
3.7317702770233154
3.720350503921509
3.736757516860962
3.6585869789123535
3.6995725631713867
3.68684983253479
3.651060104370117
3.6228413581848145
3.6582140922546387
3.6270370483398438
3.6106724739074707
3.6154656410217285
3.662477731704712
3.6274566650390625
3.6015188694000244
3.606271266937256
3.656014919281006
3.6240293979644775
3.635521650314331
3.6011760234832764
3.5746748447418213
3.564615488052368
3.5165226459503174
3.5195324420928955
3.49649977684021
2.302537679672241
2.302241325378418
2.3003296852111816
2.2817471027374268
2.1832494735717773
2.1239089965820312
2.231205701828003
2.0949864387512207
2.1246118545532227
2.028583526611328
2.0835793018341064
2.1237306594848633
2.019836664199829
2.020308017730713
2.1095588207244873
2.024553060531616
2.096480369567871
2.0285043716430664
2.027672290802002
2.047792911529541
2.0508835315704346
2.013822555541992
2.0013937950134277
1.9848649501800537
2